#### 安装python2没有的包
+ python2 -m pip install sqlalchemy
+ python2 -m pip install pyodbc

In [1]:
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
import arcpy
import time
import requests
import json

### 1.创建渔网

语法
>CreateFishnet_management (out_feature_class, origin_coord, y_axis_coord, cell_width, cell_height, number_rows, number_columns, {corner_coord}, {labels}, {template}, {geometry_type})

In [2]:
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984")
arcpy.env.overwriteOutput =  True
fc = r'D:\rui\code_analysis\file\GIS\class2203\basic_data.gdb\chengdu'
desc = arcpy.Describe(fc)
out_fc = fc + '_fishnet'
arcpy.CreateFishnet_management(out_feature_class = out_fc,
                               origin_coord = str(desc.extent.XMin) + " " + str(desc.extent.YMin),
                               y_axis_coord = str(desc.extent.XMin) + " " + str(desc.extent.YMin+3),
                               cell_width = "0.003",
                               cell_height = "0.003",
                               number_rows = "",
                               number_columns = "",
                              #corner_coord = str(desc.extent.XMax) + " " + str(desc.extent.YMax),
                               labels = "NO_LABELS",
                               template = fc,
                               geometry_type = "POLYGON")

<Result 'D:\\rui\\code_analysis\\file\\GIS\\class2203\\basic_data.gdb\\chengdu_fishnet'>

### 2.求的每个网格中心坐标

#### 2.1添加字段，用于后面字段连接

In [3]:
out_fc = fc + '_fishnet'
for i in ['check_id','lat','lng']:
    arcpy.AddField_management(out_fc,i,'TEXT') 

#### 2.2向新加字段添加值。创建一个dataframe对象，用于API检索

In [4]:
tbs = []
fields = ['OID','SHAPE@XY','check_id','lat','lng']
with arcpy.da.UpdateCursor(out_fc,fields) as cursor:
    for row in cursor:
        tb = []
        row[2] = row[0]
        row[3] = row[1][1]
        row[4] = row[1][0]
        tb.append(row[2])
        tb.append(str(row[3])+','+str(row[4]))
        tbs.append(tb)
        cursor.updateRow(row)

In [5]:
df = pd.DataFrame(tbs,columns=['check_id','coords'])
df.head(5)

,check_id,coords
0,1,"30.5582442974,103.910433099"
1,2,"30.5582442974,103.913433099"
2,3,"30.5582442974,103.916433099"
3,4,"30.5582442974,103.919433099"
4,5,"30.5582442974,103.922433099"


In [18]:
len(df)

8856

### 3.通过百度获得最短交通耗时

#### 3.1按50个坐标进行字符串组合

In [7]:
[df['coords'][50:52]]

[50    30.5582442974,104.060433099
 51    30.5582442974,104.063433099
 Name: coords, dtype: object]

In [6]:
def get_coords(data_f,data_col):
    lsts = []
    for i in range(len(data_f)/50):
        lst = list(data_f[data_col][i*50:(i+1)*50])
        lsts.append(lst)
    last_lst = list(data_f[data_col][-(len(data_f)%50):])
    lsts.append(last_lst)
    return lsts

In [7]:
coords_lst = get_coords(df,'coords')
coords_lst[-1]

['30.8012442969,104.216433099',
 '30.8012442969,104.219433099',
 '30.8012442969,104.222433099',
 '30.8012442969,104.225433099',
 '30.8012442969,104.228433099',
 '30.8012442969,104.231433099']

In [69]:
len(coords_lst)

178

#### 添加|分隔符

In [11]:
coords_lst_use = ['|'.join(i) for i in coords_lst]
coords_lst_use[-1]

'30.8012442969,104.216433099|30.8012442969,104.219433099|30.8012442969,104.222433099|30.8012442969,104.225433099|30.8012442969,104.228433099|30.8012442969,104.231433099'

#### 3.2小数据跑流程，解析百度API，返回json数据

In [9]:
def get_time(coord_org,coord_des50,api_key):
    api_addr = 'https://api.map.baidu.com/routematrix/v2/driving?origins={}&destinations={}&tactics=11&coord_type=wgs84&ak={}'\
    .format(coord_org,coord_des50,api_key)
    print(api_addr)
    req = requests.get(api_addr)
    content = req.text
    sjson = json.loads(content)
    return sjson

coord_org = '30.6636990000,104.0725260000'
coord_des50 = coords_lst_use[-1]
api_key = '7B7xW0ZIymEbzVWxvagFmg6j40ly0QtH'
dt = get_time(coord_org,coord_des50,api_key)

https://api.map.baidu.com/routematrix/v2/driving?origins=30.6636990000,104.0725260000&destinations=30.8012442969,104.216433099|30.8012442969,104.219433099|30.8012442969,104.222433099|30.8012442969,104.225433099|30.8012442969,104.228433099|30.8012442969,104.231433099&tactics=11&coord_type=wgs84&ak=7B7xW0ZIymEbzVWxvagFmg6j40ly0QtH


#### 获得各点耗时

In [96]:
[i["duration"]["value"] for i in dt["result"]]

2841

#### 3.3代码组合，进行全数据分析

In [ ]:
#坐标组合函数
def get_coords(data_f,data_col,nums):
    lsts = []
    for i in range(len(data_f) / nums):
        lst = list(data_f[data_col][i * nums:(i + 1) * nums])
        lsts.append(lst)
    last_lst = list(data_f[data_col][-(len(data_f) % nums):])
    lsts.append(last_lst)
    return lsts

#获得交通耗时，最终返回一个时间列表
#做异常判断，防止网络问题
def get_time(coord_org,coord_des,api_key):
    try:
        api_addr = 'https://api.map.baidu.com/routematrix/v2/driving?origins={}&destinations={}&tactics=11&coord_type=wgs84&ak={}'\
    .format(coord_org,coord_des,api_key)
        req = requests.get(api_addr)
        content = req.text
        sjson = json.loads(content)
        lst = [i["duration"]["value"] for i in sjson["result"]]
        return lst
    except:
        time.sleep(2)
        print('>  get_time error  <'.center(80,'='))
        get_time(coord_org,coord_des,api_key)
        
#进行坐标与时间的组合，使用zip函数，并把值转换为padas对象，存储到数据库中

def get_final_data(coord_org,coord_des,api_key,tb_name):
    try:
        get_dur_dt = get_time(coord_org,coord_des,api_key)
        time.sleep(1)
#         print(get_dur_dt)
        coords_dt = coord_des.split('|')
#         print(coords_dt)
        zip_dt = pd.DataFrame(zip(coords_dt,get_dur_dt))
        print(zip_dt.head(5))
        zip_dt.to_sql(tb_name,engine,if_exists = 'append')
    except:
        time.sleep(1)
        print('>  get_final_data error  <'.center(80,'='))
        get_final_data(coord_org,coord_des,api_key,tb_name)
        
if __name__ == "__main__":
    t1 = time.time()
    coords_lst = get_coords(df,'coords',50)
    coords_lst_use = ['|'.join(i) for i in coords_lst]
    coord_center = '30.6636990000,104.0725260000'
    api_key = '7B7xW0ZIymEbzVWxvagFmg6j40ly0QtH'
    database_name = 'traffic'
    table_name = 'traffic'
    engine = create_engine('mssql+pyodbc://:@CLAIRE/{}?driver=SQL server'.format(database_name))
    for i,coords in enumerate(coords_lst_use[:2]):
        print(i,'processing...')
        get_final_data(coord_center,coords,api_key,table_name)
    t2 = time.time()
    print('time:{} seconds'.format(t2 - t1).center(80,'='))

(0, 'processing...')
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
===========================>  get_final_data error  <===========================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
=======

==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
============================

==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
==============================>  get_time error  <==============================
============================

In [10]:
engine = create_engine('mssql+pyodbc://:@CLAIRE/test?driver=SQL server')
with engine.connect() as conn,conn.begin():
    data = pd.read_sql_table('test',conn)
data

,index,Unnamed: 0,title,url
0,0,0,如何“罩”顾好自己，这九个问题你一定要知道,http://planning.org.cn/news/view?id=10418&cid=12
1,1,1,理解城市与公共卫生的十本书,http://planning.org.cn/news/view?id=10417&cid=12
2,2,2,【图解】新型冠状病毒肺炎诊疗快速指南,http://planning.org.cn/news/view?id=10414&cid=12
3,3,3,9问新型冠状病毒,http://planning.org.cn/news/view?id=10413&cid=12
4,4,4,如何预防新型冠状病毒,http://planning.org.cn/news/view?id=10412&cid=12
5,5,5,国务院联防联控机制印发意见 分区分级精准防控,http://planning.org.cn/news/view?id=10411&cid=12
6,6,6,8项重点任务！国务院发文落实“菜篮子”市长负责制,http://planning.org.cn/news/view?id=10410&cid=12
7,7,7,李克强主持召开中央应对新冠肺炎疫情工作领导小组会议,http://planning.org.cn/news/view?id=10409&cid=12
8,8,8,习近平主持召开中央全面深化改革委员会第十二次会议,http://planning.org.cn/news/view?id=10408&cid=12
9,9,9,习近平：在中央政治局常委会会议研究应对新型冠状病毒肺炎疫情工作时的讲话,http://planning.org.cn/news/view?id=10407&cid=12
